In [18]:
import numpy as np
import librosa

ff = "wavs_single_channel/BoundaryTone-features-pyin/subj-2108_piano_1obj_mbt_zen.wav_1.wav.npy"
# load features from npy
features = np.load(ff)

In [ ]:
# get first non-zero frame
non_zero_frames = np.where(features[1] != 0)
first_non_zero_frame = non_zero_frames[0][0]
y, sr = librosa.load('wavs_single_channel/BoundaryTone/subj-2108_piano_1obj_mbt_zen.wav_1.wav', sr=44100)

print(y.shape, sr)
print(y.shape[0]/44100)
f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                            sr=16000,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(voiced_flag.shape)

f02, voiced_flag2, voiced_probs2 = librosa.pyin(y,
                                            sr=44100,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(voiced_flag2[-3:])
# print(features.shape)
# print(features.shape[1]/44100)



In [59]:
import webrtcvad
vad = webrtcvad.Vad()
vad.set_mode(2)

sample_rate = 16000
# downsampling from 44100 to 16000 and framing
y_16k = librosa.resample(y, orig_sr=sr, target_sr=sample_rate)
print(y_16k.shape)
# transfer to bytes
y_16k = np.int16(y_16k * 32768).tobytes()
print(len(y_16k))
print(y_16k[:2])
# calculate bytes of 10ms in 16000Hz
step = int(sample_rate * 0.01) * 2
# loop the bytes in 10ms frames
frames = []
for i in range(0, len(y_16k), step):
    frames.append(y_16k[i:i+step])
print(len(frames))

for i, frame in enumerate(frames):
    if vad.is_speech(frame, sample_rate):
        print(i)


(59072,)
118144
b'\xfc\xff'
370
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
313
314
315
316
317
318
319


Error: Error while processing frame